In [1]:
import uproot
import numpy as np
import sys

import scipy.signal as signal
from matplotlib.colors import LogNorm

sigProcPath = "/Users/scarpell/Desktop/ICARUS/src/signal_processing/icarus-sigproc-tools"
sys.path.insert(0,sigProcPath)

PATHNAME       = "/Users/scarpell/Desktop/ICARUS/src/signal_processing/"
PATHTODATA     = "data/noise/"
RECOFILENAME   = PATHNAME + PATHTODATA + "/data_dl1_run822_201_20200109T010140_dl3_20200128T175402-DUMMY.root"

# Below should be standard for the test data files currently available
RECOFOLDERNAME = "Events"
DAQNAME        = "raw::RawDigits_daq__DUMMY."

from sigproc_tools.sigproc_functions.noiseProcessing import getPedestalsAndRMS
from sigproc_tools.sigproc_objects.rawdigit import RawDigit
from sigproc_tools.sigproc_objects.minicrate import MiniCrate

print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)

print("Opening the folder contianing the RawDigits information: ",RECOFOLDERNAME)
events_folder = data_file[RECOFOLDERNAME]
rawdigits = RawDigit(events_folder,DAQNAME)    

Opening file:  /Users/scarpell/Desktop/ICARUS/src/signal_processing/data/noise//data_dl1_run822_201_20200109T010140_dl3_20200128T175402-DUMMY.root
Opening the folder contianing the RawDigits information:  Events


In [13]:
# Function definition for inside the loop
def _getMedianNoiseCorrection(waveforms):
        median = np.median(waveforms,axis=0)
        waveLessCoherent = waveforms - median.transpose()
        return waveLessCoherent

def _removeCoherentNoise(waveforms,grouping):
       # Define placeholders for the output arrays
        waveLessCoherent = np.array([0])

        nChannels = waveforms.shape[0]

        for idx in range(0,nChannels,grouping):
            temp = _getMedianNoiseCorrection(waveforms[idx:idx+grouping,:])
            if idx == 0:
                waveLessCoherent = temp
            else:
                waveLessCoherent = np.concatenate((waveLessCoherent,temp),axis=0)

        return waveLessCoherent


def make_planes(waveforms):
    nchannels_minicrate = 576*9
    nboard = 64
    ncables = 32
    groups = np.arange(int(nchannels_minicrate/ncables))
    plane={
        '0' : np.asarray([np.concatenate([ waveforms[e][ncables*n:ncables*(n+1)] for n in groups[(groups%2==0)] ]) for e in range(waveforms.shape[0])]),
        '1' : np.asarray([np.concatenate([ waveforms[e][ncables*n:ncables*(n+1)] for n in groups[~(groups%2==0)] ]) for e in range(waveforms.shape[0])]) }
    return plane

In [9]:
# Do the loop over the events
nEvents=10
waveforms = [] 
waveforms_cnr = [] 

for event in range(nEvents):
    print("Processing event %d" % event)
    waveform = getPedestalsAndRMS(rawdigits.getWaveforms(event))[0]
    waveforms.append(waveform)
    waveforms_cnr.append( _removeCoherentNoise(waveform, 32) )
waveforms = np.asarray(waveforms)
waveforms_cnr = np.asarray(waveforms_cnr)

Processing event 0
Processing event 1
Processing event 2
Processing event 3
Processing event 4
Processing event 5
Processing event 6
Processing event 7
Processing event 8
Processing event 9


In [ ]:
# Now study the fft per plane
plane = make_planes(waveforms)
plane_cnr = make_planes(waveforms_cnr)


# Now get the FFT
fft = {key : np.mean(signal.periodogram(item, 1.25, axis=2)[1], axis=0) for key, item in plane.items()}
fft_cnr = {key : np.mean(signal.periodogram(item, 1.25, axis=2)[1], axis=0) for key, item in plane_cnr.items()}

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2, figsize=(12,4) )
for idx, plane_label in enumerate(['0', '1']):
    out =ax[idx].imshow(fft[plane_label], vmin=0.0, vmax=200, aspect = 8)
    fig.colorbar(out, ax=ax[idx], label='Power', pad=0.02)

fig, ax = plt.subplots(1,2, figsize=(12,4) )
for idx, plane_label in enumerate(['0', '1']):
    out =ax[idx].imshow(fft_cnr[plane_label], vmin=0.0, vmax=200, aspect = 8)
    fig.colorbar(out, ax=ax[idx], label='Power', pad=0.02)

fig, ax = plt.subplots(1,2, figsize=(12,4) )
for idx, plane_label in enumerate(['0', '1']):
    ax[idx].plot(np.linspace(0.0, 1.25, fft[plane_label].shape[1] ), np.mean(fft[plane_label], axis=0), label='Noise' )
    ax[idx].plot(np.linspace(0.0, 1.25, fft_cnr[plane_label].shape[1] ), np.mean(fft_cnr[plane_label], axis=0), label='Denoised' )
    #ax[idx].plot(np.linspace(0.0, 1.25, fft_cnr[plane_label].shape[1] ), np.mean(fft[plane_label], axis=0) - np.mean(fft_cnr[plane_label], axis=0)  , label='Difference' )
    #ax[0].set_yscale('Log')
    ax[idx].set_xlim((0.0, 0.4))
    ax[idx].set_ylim((0.1, 250))
    ax[idx].set_xlabel('Frequency (MHz)')
    ax[idx].set_ylabel('Power')
    ax[idx].legend()
    
fig, ax = plt.subplots(1,2, figsize=(12,4) )
for idx, plane_label in enumerate(['0', '1']):
    #ax[idx].plot(np.linspace(0.0, 1.25, fft[plane_label].shape[1] ), np.mean(fft[plane_label], axis=0), label='Noise' )
    #ax[idx].plot(np.linspace(0.0, 1.25, fft_cnr[plane_label].shape[1] ), np.mean(fft_cnr[plane_label], axis=0), label='Denoised' )
    ax[idx].plot(np.linspace(0.0, 1.25, fft_cnr[plane_label].shape[1] ), np.mean(fft[plane_label], axis=0) - np.mean(fft_cnr[plane_label], axis=0)  , label='Difference' )
    #ax[0].set_yscale('Log')
    ax[idx].set_xlim((0.0, 0.4))
    ax[idx].set_ylim((0.1, 250))
    ax[idx].set_xlabel('Frequency (MHz)')
    ax[idx].set_ylabel('Power')
    ax[idx].legend()

plt.show()